# Single resonator


Authors: Kelvin Ramos

Description: single resonator coupled to a feedline f=6GHz, lc=100um

In [1]:
# Import useful packages
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs
from qiskit_metal.toolbox_metal import math_and_overrides
from qiskit_metal.qlibrary.core import QComponent
from collections import OrderedDict

# To create plots after geting solution data.
import matplotlib.pyplot as plt
import numpy as np

# Packages for the simple design
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.terminations.launchpad_wb_driven import LaunchpadWirebondDriven
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

# Analysis
# from qiskit_metal.renderers.renderer_gds.gds_renderer import QGDSRenderer
# from qiskit_metal.analyses.quantization import EPRanalysis
from qiskit_metal.analyses.quantization import EPRanalysis
from qiskit_metal.analyses.simulation import ScatteringImpedanceSim
from qiskit_metal.analyses.sweep_and_optimize.sweeping import Sweeping
import pyEPR as epr

# Parameters

In [2]:
from scipy.constants import c

In [3]:
### f is float in GHz
def leng(f):
    f2=f*1e9 ## to GHz
    eff=(11.9+1)/2
    le=c/(4*np.sqrt(eff)*f2)
    le=le*1e3 # to milimeters
    return print(r'resonator length l={:.4f} mm for frequency f0={} GHz'.format(le,f))
leng(6.0)

resonator length l=4.9185 mm for frequency f0=6.0 GHz


# diseño del chip

In [4]:
# Set up chip dimensions 
design = designs.DesignPlanar()
design._chips['main']['size']['size_x'] = '4mm'
design._chips['main']['size']['size_y'] = '3mm'
design._chips['main']['size']['size_z'] = '-400um-50nm'
# Resonator and feedline gap width (W) and center conductor width (S) from reference 2
design.variables['cpw_width'] = '20um' #S from reference 2
design.variables['cpw_gap'] = '11um' #W from reference 2


design.overwrite_enabled = True

hfss = design.renderers.hfss

# Open GUI
gui = MetalGUI(design)

In [5]:
design.chips.main.size['center_x'] = '2.0mm'
design.chips.main.size['center_y'] = '1.5mm'

In [6]:
# Define for renderer
eig_qres = EPRanalysis(design, "hfss")
hfss = design.renderers.hfss
hfss = eig_qres.sim.renderer
q3d = design.renderers.q3d

# Capacitors

Here we will have a single feedline couple to 16 CPW resonators.


In [104]:
leng(6)
coup=80e-3
fil=90e-3
start_leg=100e-3
len_coup=coup+fil
print(r'coupling length: {:.4f} mm'.format(len_coup))

resonator length l=4.9185 mm for frequency f0=6 GHz
coupling length: 0.1700 mm


In [105]:
#### we must to add the fillet factor to the coumpling_length


ubi_1=dict(pos_x='2.0mm', pos_y='1.5mm', prime_width='20um', prime_gap='11um', 
         second_width='20um', second_gap='11um', coupling_space='10um', 
         coupling_length='170um', down_length='100um', fillet='90um', hfss_wire_bonds=True)

Capacitor_1 = CoupledLineTee(design, 'Capacitor_1', options=ubi_1)

gui.rebuild()
gui.autoscale()

# Launchpad


The lauchpad should be included in the driven model simulations.

For that reason, we use the LaunchpadWirebondDriven component which has an extra pin for input/output

In [106]:
###################
# Single feedline #
###################

# Driven Lauchpad 1
x1 = '0.5mm'
y1 = '1.5mm'
ops_1 = Dict(chip='main', pos_x=x1, pos_y=y1, orientation='360', lead_length='10um', 
             pad_width='120um', pad_gap='61um', trace_width='20um', trace_gap='11um')
LP1 = LaunchpadWirebondDriven(design, 'LP1', options = ops_1)

# Driven Launchpad 2
x2 = '3.5mm'
y2 = '1.5mm'
ops_2 = Dict(chip='main', pos_x=x2, pos_y=y2, orientation='180', lead_length='10um',
             pad_width='120um', pad_gap='61um', trace_width='20um', trace_gap='11um')
LP2 = LaunchpadWirebondDriven(design, 'LP2', options = ops_2)


# Rebuild the GUI
gui.rebuild()
gui.autoscale()

# Transmission lines

## First part

In [107]:
# Using path finder to connect the two launchpads
# Using path finder to connect the two launchpads
#### LP1 capacitor
linea_1 = RoutePathfinder(design, 'linea_1', options = Dict(chip='main', trace_width ='20um',
        trace_gap ='11um', fillet='10um', hfss_wire_bonds = True, lead=Dict(end_straight='0.0mm'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='LP1',
                                                    pin='tie'),
                                                end_pin=Dict(
                                                    component='Capacitor_1',
                                                    pin='prime_start')
                                    
        )))
linea_4 = RoutePathfinder(design, 'linea_4', options = Dict(chip='main', trace_width ='20um',
        trace_gap ='11um', fillet='10um', hfss_wire_bonds = True,
                                            lead=Dict(end_straight='0.0mm'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Capacitor_1',
                                                    pin='prime_end'),
                                                end_pin=Dict(
                                                    component='LP2',
                                                    pin='tie')
                                            )))
#### LP2 capacitorb


# Rebuild the GUI
gui.rebuild()
gui.autoscale() 


INFO 04:23PM [__del__]: Disconnected from Ansys HFSS
INFO 04:23PM [__del__]: Disconnected from Ansys HFSS
INFO 04:23PM [__del__]: Disconnected from Ansys HFSS
INFO 04:23PM [__del__]: Disconnected from Ansys HFSS
INFO 04:23PM [__del__]: Disconnected from Ansys HFSS


# Resonators

#### coupling length

In [108]:
leng(6.0)
# len_true=5.902-coup-fil-start_leg
len_true=4.9185-coup-fil
print('true_length', len_true)

resonator length l=4.9185 mm for frequency f0=6.0 GHz
true_length 4.7485


In [109]:
######################
# lambda/4 resonator #
######################

otg1 = ShortToGround(design, 'otg1', options=Dict(chip='main', pos_x='2.5mm',  pos_y='0.25mm',
                                                  orientation='0'))

# Use RouteMeander to fix the total length of the resonator
meandro_1 = RouteMeander(design, 'meandro_1',  options=Dict(chip='main',
        trace_width ='cpw_width',
        trace_gap ='11um',
        total_length='4.7485mm',
        hfss_wire_bonds = True,
        fillet='90 um',
        lead = Dict(start_straight='400um', end_straight='100um'),
        meander=Dict(spacing='260um', asymmetry='0um'),
        pin_inputs=Dict(
            start_pin=Dict(component='Capacitor_1', pin='second_end'),
            end_pin=Dict(component='otg1', pin='short')), ))

# rebuild the GUI
gui.rebuild()
gui.autoscale()

In [93]:
meandro_1.options

{'chip': 'main',
 'layer': '1',
 'pin_inputs': {'start_pin': {'component': 'Capacitor_1', 'pin': 'second_end'},
  'end_pin': {'component': 'otg1', 'pin': 'short'}},
 'fillet': '90 um',
 'lead': {'start_straight': '400um',
  'end_straight': '100um',
  'start_jogged_extension': '',
  'end_jogged_extension': ''},
 'total_length': '4.6285mm',
 'trace_width': 'cpw_width',
 'meander': {'spacing': '260um', 'asymmetry': '0um'},
 'snap': 'true',
 'prevent_short_edges': 'true',
 'hfss_wire_bonds': True,
 'q3d_wire_bonds': False,
 'aedt_q3d_wire_bonds': False,
 'aedt_hfss_drivenmodal_wire_bonds': False,
 'aedt_hfss_eigenmode_wire_bonds': False,
 'trace_gap': '11um',
 '_actual_length': '4.6285 mm'}

# Simulacion

### EPR analysis for frequency convergence

In [94]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_res = EPRanalysis(design, "hfss")

In [18]:
hfss = eig_res.sim.renderer

In [19]:
hfss.start() ## start hfss

INFO 01:32PM [connect_project]: Connecting to Ansys Desktop API...
INFO 01:32PM [load_ansys_project]: 	Opened Ansys App
INFO 01:32PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 01:32PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/Quantum Circuits IB/Documents/Kelvin/qiskit-metal/qmetal/simu/resonator/projects_hfss/
	Project:   reso4Ghz
INFO 01:32PM [connect_design]: 	Opened active design
	Design:    Readout3 [Solution type: Eigenmode]
INFO 01:32PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 01:32PM [connect]: 	Connected to project "reso4Ghz" and design "Readout3" 😀 



True

In [20]:
# clean the design if needed
hfss.clean_active_design()

In [21]:
hfss.activate_ansys_design("Readout3", 'eigenmode')  # use new_ansys_design() to force creation of a blank design

INFO 01:32PM [connect_design]: 	Opened active design
	Design:    Readout3 [Solution type: Eigenmode]


In [22]:
hfss.render_design([], [])
#hfss.save_screenshot()

In [23]:
# Analysis properties
setup = hfss.pinfo.setup
setup.passes = 8
setup.delta_f = 0.01
# print(f"""
# Number of eigenmodes to find             = {setup.n_modes}
# Number of simulation passes              = {setup.passes}
# Convergence freq max delta percent diff  = {setup.delta_f}
# """)

# Next 2 lines are counterinuitive, since there is no junction in this resonator.
# However, these are necessary to make pyEPR work correctly. Please do note delete
hfss.pinfo.design.set_variable('Lj', '10 nH')
hfss.pinfo.design.set_variable('Cj', '0 fF')
setup.analyze()

INFO 01:33PM [analyze]: Analyzing setup Setup


In [24]:
eig_res.sim.convergence_t, eig_res.sim.convergence_f, _ = hfss.get_convergences()
eig_res.sim.plot_convergences()

01:33PM 58s INFO [get_f_convergence]: Saved convergences to C:\Users\Quantum Circuits IB\Documents\Kelvin\qiskit-metal\qmetal\simu\resonator\code_qm\hfss_eig_f_convergence.csv


### Driven modal for S21 behaviour

In [110]:
from qiskit_metal.analyses.simulation import ScatteringImpedanceSim
em1 = ScatteringImpedanceSim(design, "hfss")


In [111]:
design_name= "Sweep_DrivenModal"
qcomp_render = [] # Means to render everything in qgeometry table.
open_terminations = []

# Here, pin LP1_in and LP2_in are converted into lumped ports,
#           each with an impedance of 50 Ohms. <br>
port_list = [('LP1', 'in', 50),
             ('LP2', 'in', 50)]
box_plus_buffer = True

In [112]:
# we use HFSS as rendere
hfss = em1.renderer
hfss.start()

INFO 04:23PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:23PM [load_ansys_project]: 	Opened Ansys App
INFO 04:23PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 04:23PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/Quantum Circuits IB/Documents/Kelvin/qiskit-metal/qmetal/simu/resonator/code_qm/6GHz/
	Project:   Project6GHz
INFO 04:23PM [connect_design]: 	Opened active design
	Design:    freq_6ghz [Solution type: DrivenModal]
INFO 04:23PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)
INFO 04:23PM [connect]: 	Connected to project "Project6GHz" and design "freq_6ghz" 😀 



True

In [113]:
# Here we activate the design for a drivenmodal solution
hfss.activate_ansys_design("freq_6ghz", 'drivenmodal')
setup_args = Dict(max_delta_s=0.01, freq_ghz=6, max_passes=10)
setup_args.name = 'Setup'
hfss.edit_drivenmodal_setup(setup_args)

INFO 04:23PM [connect_design]: 	Opened active design
	Design:    freq_6ghz [Solution type: DrivenModal]
04:23PM 37s WARNING [edit_drivenmodal_setup]: In setup_args, key=max_delta_s, value=0.01 is not in pinfo.setup, the key/value pair from setup_args not added to Setup in Ansys.
04:23PM 37s WARNING [edit_drivenmodal_setup]: The value for freq_ghz should be an float. The present value is 6.
04:23PM 37s WARNING [edit_drivenmodal_setup]: In setup_args, key=freq_ghz, value=6 is not in pinfo.setup, the key/value pair from setup_args not added to Setup in Ansys.


In [114]:
# set buffer: espacio alrededor de la geometría que está siendo simulada
hfss.options['x_buffer_width_mm'] = 0.4
hfss.options['y_buffer_width_mm'] = 0.4

In [115]:
# clean the design if needed
hfss.clean_active_design()

In [116]:
# render the design
hfss.render_design(selection=[], 
                   open_pins=open_terminations, 
                   port_list=port_list, 
                   box_plus_buffer = box_plus_buffer)

In [117]:
hfss.modeler.mesh_length(
                'cpw_mesh',
                ['trace_meandro_1', 'second_cpw_Capacitor_1'],
                MaxLength='0.008mm')

In [103]:
## usando un step se recomienda que sea 1/10 el ancho de banda.
## para una simulacion, el sweep debe contener a la frecuencia de resonancia
hfss.add_sweep(setup_name="Setup", 
               name="Sweep", 
               start_ghz=5.8,
               stop_ghz=6.2,
               count=600,
               type="Fast")

INFO 03:38PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)


# GDS file

In [48]:
a_gds = design.renderers.gds
### import and place the cell names in the Fake_Junctions
# a_gds.options['path_filename'] = '../Files_mascara/Fake_Junctions.GDS'
a_gds.options['path_filename'] = '../Fake_Junctions.GDS'

In [49]:
### import and place the cell names in the Fake_Junctions
# a_gds.options['path_filename'] = '../Files_mascara/Fake_Junctions.GDS'

In [50]:
## for fillet the short segments, True means no fillet
a_gds.options['short_segments_to_not_fillet'] = 'False'

scale_fillet = 2.0
a_gds.options['check_short_segments_by_scaling_fillet'] = scale_fillet

In [51]:
# Restore a_gds options

a_gds.options.no_cheese['view_in_file']['main']={1: False}
a_gds.options.cheese['view_in_file']['main']={1: False}
a_gds.options['max_points'] = '8191'
a_gds.options['no_cheese']['buffer']='50um'
# We will shift the center of the chip to origin

design.chips.main.size['center_x'] = '2.0mm'
design.chips.main.size['center_y'] = '1.5mm'
design.rebuild()

### For demo, set max_points to 8191 and look at the GDS output.
a_gds.options['max_points'] = '8191'

# We next export it to GDS

design.renderers.gds.export_to_gds("res6G30lc_gds.gds")

1

In [30]:
#Many ways to view the QGeometry tables.
#If you want to view, uncomment below lines and and run it.

design.qgeometry.tables
# design.qgeometry.tables['path']
# design.qgeometry.tables['poly']

{'path':     component            name  \
 0           1       prime_cpw   
 1           1   prime_cpw_sub   
 2           1      second_cpw   
 3           1  second_cpw_sub   
 4           2       prime_cpw   
 ..        ...             ...   
 125        65             cut   
 126        66           trace   
 127        66             cut   
 128        67           trace   
 129        67             cut   
 
                                               geometry  layer  subtract  \
 0        LINESTRING (1.37500 9.00000, 1.62500 9.00000)      1     False   
 1        LINESTRING (1.37500 9.00000, 1.62500 9.00000)      1      True   
 2    LINESTRING (1.43750 8.94700, 1.56250 8.94700, ...      1     False   
 3    LINESTRING (1.42650 8.94700, 1.56250 8.94700, ...      1      True   
 4        LINESTRING (3.38800 9.00000, 3.61200 9.00000)      1     False   
 ..                                                 ...    ...       ...   
 125  LINESTRING (4.44400 1.15300, 4.44400 1.55300